In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt

In [2]:
keys = ["Maruti", "Hyundai", "Honda", "Renault", "Tata", "Datsun", "Volkswagen", "Ford", "KIA", "Skoda", "Jeep", "Volvo"]
cities = ["hyderabad","Ahmedabad","Bangalore","Chennai","Indore"]
base_url = "https://www.cars24.com/buy-used-{}-cars-{}/"

lst_urls = []

for city in cities:
    for key in keys:
        url = base_url.format(key, city)
        lst_urls.append(url)

#lst_urls


In [9]:
# empty lists to save data after scrapping
years = []
brand = []
model = []
km = []  
owner=[]
varient=[]
num_plate=[]
bmh = []
emi = []
fin_price=[]
act_price=[]
transmission=[]
location=[]



      

for url in lst_urls:
    page = requests.get(url)
    soup=BeautifulSoup(page.text)




    raw_data=soup.find_all("h2", class_="_2lmIw")
    for i in raw_data:
        a = i.text
        years.append(a.split(" ")[0])
        brand.append(a.split(" ")[1])
        model.append(a.split(" ")[-1])
        

    detail=soup.find_all("ul", class_="_13yb6")

    for i in detail:
        km.append(i.text.split(" ")[0])        
        v = (i.text.split(" ")[2].split("Owner")[1])
        vv=(re.findall(r"(Petrol|CNG|Diesel|LPG)",v))
        varient.append(" ".join(vv))
        
        
    for i in detail:
        oo=i.text.split(" ")[1].split("km")[1]
        o=re.findall("(\d)\w+",oo)
        owner.append(" ".join(o))

    lst=["Petrol","CNG","Diesel","LPG"]
    
    for i in detail:
        a=(i.text.split("Owner")[1])
        for key in lst:
            if key in a:
                num_plate.append(i.text.split(key,1)[1].strip(" "))


    range_version=soup.find_all("ul", class_="_1hOnS")

    for i in range_version:
        bmh.append(i.text.split("Manual")[0])


    monthly=soup.find_all("div", class_="_1Em-A")

    for i in monthly:
        emi.append(i.text.split("/month")[0])

    f_price=soup.find_all("div", class_="_18ToE")


    for i in f_price:
        v = (i.text.split("₹")[1])
        fin_price.append(v)


    t_price=soup.find_all("div", class_="_18ToE")

    for i in t_price:
        v = (i.text.split("₹")[-1])    
        act_price.append(v)


    g=soup.find_all("ul", class_="_1hOnS")

    for i in g:
        v = (i.text.split(" ")[-1])
        vv = (re.findall(r"(Manual|Automatic)",v))
        transmission.append(" ".join(vv))





    loc=soup.find_all("span", class_="_3DYbK")    



    for i in loc:
        l = (i.text.split("\n")[0].strip("."))
        location.append(l)

In [10]:
print(len(years))
print(len(brand))
print(len(model))
print(len(km))
print(len(owner))
print(len(varient))
print(len(num_plate))
print(len(bmh))
print(len(emi))
print(len(fin_price))
print(len(act_price))
print(len(transmission))
print(len(location))

833
833
833
833
833
833
833
833
833
833
833
833
833


In [11]:
dict={"MODELS":model,"VERSION":bmh,"BRANDS":brand,"Fuel":varient,"MODEL_YEAR":years,"RTO":num_plate,
      "Transmission":transmission,"OWNER":owner,"KM_DRIVEN":km,"Location":location,"EMI":emi,
      "FINAL_PRICE":fin_price,"ACTUAL_PRICE":act_price}

In [12]:
df=pd.DataFrame(dict)
df

,MODELS,VERSION,BRANDS,Fuel,MODEL_YEAR,RTO,Transmission,OWNER,KM_DRIVEN,Location,EMI,FINAL_PRICE,ACTUAL_PRICE
0,Dzire,VXI,Maruti,Petrol,2021,TS-15,Manual,1,"17,294","Kompally, Hyderabad","₹13,470","6,89,000","6,89,000"
1,Ertiga,ZXI,Maruti,Petrol,2015,TS-08,Manual,1,"96,184","Kompally, Hyderabad","₹12,629","6,46,000","6,74,000"
2,Dzire,VXI,Maruti,Petrol,2020,TS-10,Manual,1,"88,126","Kompally, Hyderabad","₹13,138","6,72,000","7,20,000"
3,Baleno,ZETA CVT PETROL 1.2Automatic,Maruti,Petrol,2021,TS-13,Automatic,1,"45,356","Bachupally, Hyderabad","₹15,912","8,36,000","8,36,000"
4,Celerio,ZXI (O),Maruti,Petrol,2019,TS-34,Manual,2,"84,161","Attapur, Hyderabad","₹9,912","5,07,000","5,39,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
828,CARENS,LUXURY 1.5D MT 7STR,KIA,Diesel,2022,MP-09,Manual,1,"24,000","Vijay Nagar, Indore","₹29,404","15,44,800","16,66,000"
829,Rapid,STYLE 1.6 MPI ATAutomatic,Skoda,Petrol,2018,MP-09,Automatic,1,"70,893","Dewas Naka, Lasudia Mori","₹15,503","7,93,000","8,76,000"
830,Octavia,STYLE 2.0 TDI MT,Skoda,Diesel,2018,MP-09,Manual,1,"95,593","Dewas Naka, Lasudia Mori","₹23,412","12,30,000","14,92,000"
831,Rapid,AMBITION 1.6 MPI MT,Skoda,Petrol,2018,MP-09,Manual,1,"33,510","Dewas Naka, Lasudia Mori","₹11,574","5,92,000","6,59,000"


In [7]:
df.to_csv("cars24.csv")

In [1]:
df["Location"].unique()

NameError: name 'df' is not defined